<a href="https://colab.research.google.com/github/Nostal1ga/8959prgram_prepare/blob/main/pyg_tut/TUT3_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

Structure

In [6]:
#Base case
class GATLayer(nn.Module):
  def __init__(self):
    super(GATLayer, self).__init__()#initialize the class, call "super" specifying your aggregation

  def forward(self, input, adj):
    print(" ")

In [8]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features,out_features)))
nn.init.xavier_uniform_(W.data,gain=1.414)
input = torch.rand(nb_nodes,in_features)

h = torch.mm(input,W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [9]:
a = nn.Parameter(torch.zeros(size=(2*out_features,1)))
nn.init.xavier_uniform_(a.data,gain = 1.414)
print(a.shape)

leakyrelu = nn.LeakyReLU(0.2)

torch.Size([4, 1])


In [11]:
a_input = torch.cat([h.repeat(1,N).view(N*N,-1), h.repeat(N,1)], dim=1).view(N,-1,2*out_features)

In [12]:
e = leakyrelu(torch.matmul(a_input,a).squeeze(2))

In [14]:
print(a_input.shape, a.shape)
print("")
print(torch.matmul(a_input,a).shape)
print("")
print(torch.matmul(a_input,a).squeeze(2).shape)

torch.Size([3, 3, 4]) torch.Size([4, 1])

torch.Size([3, 3, 1])

torch.Size([3, 3])


Masked Attention

In [15]:
# Masked Attention
adj = torch.randint(2,(3,3))
zero_vec = -9e15*torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [16]:
attention = torch.where(adj>0, e, zero_vec)
print(adj, "\n", e, "\n", zero_vec)
attention

tensor([[1, 0, 1],
        [0, 0, 0],
        [0, 1, 1]]) 
 tensor([[-0.2220, -0.0950,  0.2007],
        [-0.3181, -0.1911, -0.0559],
        [-0.4623, -0.3354, -0.2002]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]])


tensor([[-2.2200e-01, -9.0000e+15,  2.0066e-01],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -3.3538e-01, -2.0021e-01]], grad_fn=<WhereBackward0>)

In [17]:
attention = F.softmax(attention, dim =1)
h_prime = torch.matmul(attention,h)

In [18]:
attention

tensor([[0.3959, 0.0000, 0.6041],
        [0.3333, 0.3333, 0.3333],
        [0.0000, 0.4663, 0.5337]], grad_fn=<SoftmaxBackward0>)

In [19]:
h_prime

tensor([[ 2.4552, -1.4964],
        [ 2.5126, -1.3958],
        [ 2.2673, -1.4275]], grad_fn=<MmBackward0>)

Build a layer

In [20]:
class GATLayer(nn.Module):#full step to initlialize the layer
  def __init__(self, in_features, out_features, dropout,alpha, concat=True):
    super(GATLayer,self).__init__()
    self.dropout = dropout
    self.in_features = in_features
    self.out_features = out_features
    self.alpha = alpha
    self.concat = concat

    #Xavier Initialization of weights
    self.W = nn.Parameter(torch.zeros(size=(in_features,out_features)))
    nn.init.xavier_uniform_(self.W.data, gain =1.414)

    self.a = nn.Paramter(torch.zeros(size=(2*out_features,1)))
    nn.init.xavier_uniform_(self.a.data, gain=1.414)